The first notebook to test the idea. 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

import kgof
import kgof.data as data
import kgof.density as density
import kgof.goftest as gof
import kgof.kernel as kernel
import kgof.util as util
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

In [ ]:
# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 18
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

## Isotropic normal distribution

In [ ]:
d = 1
mean = np.zeros(d)
variance = 5
isonorm = density.IsotropicNormal(mean, variance)

In [ ]:
X = np.linspace(-8, 8, 200)[:, np.newaxis]
#X = np.random.randn(100, d)
dens = isonorm.log_den(X)
grads = isonorm.grad_log(X)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(X, dens, 'b-', label='log den')
plt.plot(X, grads, 'r-', label='gradient')
plt.grid()
plt.legend(loc='best')

## Test FSSD 

In [ ]:
# true p
seed = 12
d = 50
mean = np.zeros(d)
variance = 1
isonorm = density.IsotropicNormal(mean, variance)

In [ ]:
# sample
n = 500

# only one dimension of the mean is shifted
draw_mean = mean + np.hstack((1, np.zeros(d-1)))
#draw_mean = mean +1
draw_variance = variance + 0
X = util.randn(n, d, seed=seed)*np.sqrt(draw_variance) + draw_mean
dat = data.Data(X)

In [ ]:
# Test
J = 3
alpha = 0.01

sig2 = util.meddistance(X, subsample=1000)**2
k = kernel.KGauss(sig2)

# random test locations
V = util.fit_gaussian_draw(X, J, seed=seed+1)
fssd = gof.FSSD(isonorm, k, V, alpha=alpha, n_simulate=2000, seed=2)

In [ ]:
test_result = fssd.perform_test(dat, return_simulated_stats=True)
test_result

In [ ]:
sim_stats = test_result['sim_stats']
plt.hist(sim_stats, bins=20, normed=True);
plt.stem([test_result['test_stat']], [0.03], 'r-o', label='Stat')
plt.legend()

## Optimized Gaussian FSSD

In [ ]:
# true p
seed = 20
d = 3
mean = np.zeros(d)
variance = 1
isonorm = density.IsotropicNormal(mean, variance)

In [ ]:
# sample
n = 500

# only one dimension of the mean is shifted
#draw_mean = mean + np.hstack((1, np.zeros(d-1)))
draw_mean = mean +0
draw_variance = variance + 1
X = util.randn(n, d, seed=seed)*np.sqrt(draw_variance) + draw_mean
dat = data.Data(X)
tr, te = dat.split_tr_te(tr_proportion=0.5, seed=seed+1)

In [ ]:
# Test
J = 5
alpha = 0.01

gwidth0 = util.meddistance(X, subsample=1000)**2
# random test locations
V0 = util.fit_gaussian_draw(X, J, seed=seed+1)

In [ ]:
k0 = kernel.KGauss(gwidth0)
fssd = gof.FSSD(isonorm, k0, V0, alpha=alpha, n_simulate=2000, seed=3)
fssd.perform_test(te)

In [ ]:
fssd.get_H1_mean_variance(te)

----------------

In [ ]:
opts = {
    'reg': 1e-2,
    'max_iter': 50, 
    'tol_fun':1e-3, 
    'disp':True
}
V_opt, gw_opt, opt_result = gof.GaussFSSD.optimize_locs_widths(isonorm, tr, gwidth0, V0, **opts)

In [ ]:
gw_opt

In [ ]:
# construct a test
k_opt = kernel.KGauss(gw_opt)
fssd_opt = gof.FSSD(isonorm, k_opt, V_opt, alpha=alpha, n_simulate=2000, seed=3)
fssd_opt_result = fssd_opt.perform_test(te, return_simulated_stats=True)
fssd_opt_result

In [ ]:
# get the mean and variance under H1 of the test statistic
fssd_opt.get_H1_mean_variance(te)

In [ ]:
sim_stats = fssd_opt_result['sim_stats']
plt.hist(sim_stats, bins=20, normed=True);
plt.stem([fssd_opt_result['test_stat']], [0.03], 'r-o', label='Stat')
plt.legend()

In [ ]:
gof.GaussFSSD.optimize_auto_init(isonorm, tr, J, **opts)